In [11]:
import findspark
findspark.init()

In [12]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of x second
spark=SparkSession.builder.appName("NetworkWordCount").master("local[2]").getOrCreate()
ssc = StreamingContext(spark.sparkContext, 5)

In [13]:
# Create a DStream that will connect to hostname:port, like localhost:9999
# We can use netcat to simulate the stream : nc -lk 9998
lines = ssc.socketTextStream("localhost", 9999)

In [14]:
lines

In [15]:
# Split each line into words
words = lines.flatMap(lambda line: line.split())

In [16]:
words

In [8]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [17]:
count = words.count()


In [18]:
help(count.saveAsTextFiles)

Help on method saveAsTextFiles in module pyspark.streaming.dstream:

saveAsTextFiles(prefix, suffix=None) method of pyspark.streaming.dstream.TransformedDStream instance
    Save each RDD in this DStream as at text file, using string
    representation of elements.



In [19]:
count.saveAsTextFiles("save/count")

In [20]:
ssc.start()             # Start the computation
#ssc.awaitTermination()  # Wait for the computation to terminate

In [21]:
ssc.stop()